In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random 
import time
from scipy.stats import rankdata

In [2]:
def TDT(n):
    b = n[0] + n[2] + 2*n[3]
    c = n[1] + n[2] + 2*n[4]
    
    if b == 0 and c == 0:
        return 0
    elif b == 0:
        return 1000000000
    else:
        return (b-c)**2/(b+c)

In [3]:
def lap_normal(stats, s, epsilon, K, m):
    g = sorted(stats, reverse=True)
    pri_data = g + np.random.laplace(loc = 0.0, scale = (2*K*s)/epsilon, size=np.size(stats))
    pri_r = np.argsort(pri_data)
    
    return pri_r[m-K:m]

In [4]:
def random_num(pd):
    dist = np.cumsum(pd).tolist()
    dist[-1] = 1.0
    num = np.random.rand()
    dist.append(num)
    return sorted(dist).index(num)

def exp_stats(stats, s, epsilon, K, m):
    g = sorted(stats, reverse=True)
    
    h = np.zeros(m)
    for i in range(m):
        h[i] = g[i]
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * h[i] / (2*K*s))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        h[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [5]:
def SHD(cc, n):
    T = TDT(n)
    
    d = np.zeros(2)
    N = np.zeros(6)
    
    for k in range(6):
        N[k] = n[k]
    
    if T < cc:
        while T < cc:
            if N[4] > 0:
                N[4] -= 1
            elif N[1] > 0:
                N[1] -= 1
            elif N[2] > 0:
                N[2] -= 1
            elif N[5] > 0:
                N[5] -= 1
            else:
                N[0] -= 1
            N[3] += 1
            
            T = TDT(N)
            d[0] -= 1
        
        T = TDT(n)
        for k in range(6):
            N[k] = n[k]
        
        while T < cc:
            if N[3] > 0:
                N[3] -= 1
            elif N[0] > 0:
                N[0] -= 1
            elif N[2] > 0:
                N[2] -= 1
            elif N[5] > 0:
                N[5] -= 1
            else:
                N[1] -= 1
            N[4] += 1
            
            T = TDT(N)
            d[1] -= 1
        
        if d[0] > d[1]:
            return d[0]
        else:
            return d[1]
    
    else:
        step = 0
        
        if n[0] + 2*n[3] > n[1] + 2*n[4]:
            while T >= cc:
                if n[3] > 0:
                    n[3] -= 1
                elif n[0] > 0:
                    n[0] -= 1
                elif n[5] > 0:
                    n[5] -= 1
                elif n[2] > 0:
                    n[2] -= 1
                else:
                    n[1] -= 1
                n[4] += 1
            
                T = TDT(n)
                step += 1
        else:
            while T >= cc:
                if n[4] > 0:
                    n[4] -= 1
                elif n[1] > 0:
                    n[1] -= 1
                elif n[5] > 0:
                    n[5] -= 1
                elif n[2] > 0:
                    n[2] -= 1
                else:
                    n[0] -= 1
                n[3] += 1
            
                T = TDT(n)
                step += 1
        
        return step-1

def exp_shd(n, epsilon, K, m):
    shd = np.zeros(m)
    cc = 29.7
    
    for i in range(m):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5])]
        shd[i] = SHD(cc,k)
    
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * shd[i] / (2*K))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        shd[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [6]:
def FPA(stats, sensitivity, epsilon, K, m):
    rank = np.argsort(stats)
    g = np.zeros(m)
    for i in range(m):
        g[i] = stats[i]
        
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    pri_x = np.zeros(m)
    
    s = int(m/(2*K))
    
    l = 2*K*sensitivity*np.sqrt(s)/epsilon
    
    prime_y[0:s] = y[0:s] + np.random.laplace(loc= 0.0, scale = l, size = s)
    
    pri_x = np.fft.ifft(prime_y).real
    
    pri_r = np.argsort(pri_x)
    
    #for i in range(K):
    #    for j in range(m):
    #        if pri_r[m-i-1] == rank[m-j-1]:
    #            pri_r[m-i-1] = j
    #            break
    
    return pri_r[m-K:m]

In [7]:
def extended_FPA(stats, sensitivity, epsilon, K, m):
    g = sorted(stats, reverse=True)
    
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    
    s = int(m/(2*K))
    
    for i in range(s):
        if i == 0:
            prime_y[0] = y[0]
        else:
            prime_y[i] = y[i]
            prime_y.real[int(M)-i] = prime_y.real[i]
            prime_y.imag[int(M)-i] = -prime_y.imag[i]
    
    #prime_g = np.fft.ifft(prime_y)
    
    l = 2*K*sensitivity/epsilon
    
    pri_x = np.zeros(m)
    
    noise = np.zeros(m)
    
    for j in range(m):
        hat_y = np.fft.fft(zeros)
        p = np.random.laplace(loc= 0.0, scale = l, size = 1)
        hat_y.real[0] = prime_y.real[0] + p
        for k in range(1,s):
            hat_y.real[k] = prime_y.real[k] + p * math.cos(-2*j*k*math.pi/m)
            hat_y.imag[k] = prime_y.imag[k] + p * math.sin(-2*j*k*math.pi/m)
            hat_y.real[m-k] = hat_y.real[k]
            hat_y.imag[m-k] = -hat_y.imag[k]
            
        pri_x[j] = np.fft.ifft(hat_y).real[j]
        
    pri_r = np.argsort(pri_x)
    
    return pri_r[m-K:m]

In [8]:
def lap_DFT(stats, sensitivity, epsilon, K, m):
    g = sorted(stats, reverse=True)
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    
    s = int(m/(2*K))
    
    for i in range(s):
        if i == 0:
            prime_y[0] = y[0]
        else:
            prime_y[i] = y[i]
            prime_y.real[int(M)-i] = prime_y.real[i]
            prime_y.imag[int(M)-i] = -prime_y.imag[i]
    
    prime_g = np.fft.ifft(prime_y)
    
    pri_g = prime_g.real + np.random.laplace(loc = 0.0, scale = (2*K)*((2*s-1)/m)*sensitivity/epsilon, size=m)
    
    pri_r = np.argsort(pri_g)
    
    return pri_r[m-K:m]

In [9]:
def exp_DFT(stats, sensitivity, epsilon, K, m):
    g = sorted(stats, reverse=True)
    zeros = np.zeros(m)
    y = np.fft.fft(g)
    prime_y = np.fft.fft(zeros)
    
    s = int(m/(2*K))
    
    for i in range(s):
        if i == 0:
            prime_y[0] = y[0]
        else:
            prime_y[i] = y[i]
            prime_y.real[int(m)-i] = prime_y.real[i]
            prime_y.imag[int(m)-i] = -prime_y.imag[i]
    
    prime_g = np.fft.ifft(prime_y).real
    
    h = np.zeros(m)
    for i in range(m):
        h[i] = prime_g[i]
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp((epsilon * h[i] / (2*K*sensitivity)) * (m / (2*s-1)))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
            
        x = random_num(p)
        h[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [10]:
def runTime(K, N, M):
    S = np.zeros(M)
    b = np.zeros(M)
    c = np.zeros(M)
    
    n = np.zeros((M,6))
    
    sig = random.sample(range(M),10)
    
    for i in range(10):
        S[sig[i]] = random.randint(0,2*N)
        b[sig[i]] = np.random.binomial(S[sig[i]],0.56)
        c[sig[i]] = S[sig[i]] - b[sig[i]]
        n[sig[i]][0] = b[sig[i]]
        n[sig[i]][1] = c[sig[i]]
        n[sig[i]][5] = 2*N - b[sig[i]] - c[sig[i]]
    
    for i in range(M):
        if S[i] == 0:
            S[i] = random.randint(0,2*N)
            b[i] = np.random.binomial(S[i],0.5)
            c[i] = S[i] - b[i]
            n[i][0] = b[i]
            n[i][1] = c[i]
            n[i][5] = 2*N - b[i] - c[i]
        
    stats = np.zeros(M)
    order = np.zeros(M)
    
    for i in range(M):
        order[i] = i
        if b[i] == 0 and c[i] == 0:
            stats[i] = 0
        else:
            stats[i] = (b[i]-c[i])**2/(b[i]+c[i])
    
    x = np.argsort(stats)
    #for i in range(10):
    #    print(x[M-1-i])
    
    nn = np.zeros((M,6))
    for i in range(M):
        nn[i] = n[x[M-1-i]]
    
    sensitivity = 8*(N-1)/N
        
    RT = np.zeros(7)
    
    start = time.time()
    lap_normal(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[0] = end-start
    
    start = time.time()
    exp_stats(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[1] = end-start
    
    start = time.time()
    exp_shd(nn, 3, K, M)
    end = time.time()
    RT[2] = end-start
    
    start = time.time()
    FPA(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[3] = end-start
    
    #start = time.time()
    #extended_FPA(stats, sensitivity, 3, K, M)
    #end = time.time()
    #RT[4] = end-start
    
    start = time.time()
    lap_DFT(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[5] = end-start
    
    start = time.time()
    exp_DFT(stats, sensitivity, 3, K, M)
    end = time.time()
    RT[6] = end-start
    
    return RT

In [11]:
#K = 1

N = 5000
M = 1000000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(1,5000,1000000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    #print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
#print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 1.0962028503417969 [sec]
exp_stats_time = 2.0427939891815186 [sec]
exp_shd_time = 1037.349752664566 [sec]
FPA_time = 0.48806190490722656 [sec]
lap_DFT_time = 1.8023381233215332 [sec]
exp_DFT_time = 3.168870210647583 [sec]
------------------
lap_normal_time = 1.0767149925231934 [sec]
exp_stats_time = 2.081723213195801 [sec]
exp_shd_time = 1069.086394071579 [sec]
FPA_time = 0.45689916610717773 [sec]
lap_DFT_time = 1.786931037902832 [sec]
exp_DFT_time = 3.1099488735198975 [sec]
------------------
lap_normal_time = 1.0861308574676514 [sec]
exp_stats_time = 2.074643135070801 [sec]
exp_shd_time = 1057.2354140281677 [sec]
FPA_time = 0.465770959854126 [sec]
lap_DFT_time = 1.7836809158325195 [sec]
exp_DFT_time = 3.145130157470703 [sec]
------------------
lap_normal_time = 1.0747549533843994 [sec]
exp_stats_time = 2.0809900760650635 [sec]
exp_shd_time = 1059.1542930603027 [sec]
FPA_time = 0.4691340923309326 [sec]
lap_DFT_time = 1.7771501541137695 [sec]
exp_DFT_time = 3.13275122

In [12]:
#K = 3

N = 5000
M = 1000000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(3,5000,1000000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    #print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
#print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 1.0862500667572021 [sec]
exp_stats_time = 4.293689012527466 [sec]
exp_shd_time = 1079.5795998573303 [sec]
FPA_time = 0.4880030155181885 [sec]
lap_DFT_time = 1.357314109802246 [sec]
exp_DFT_time = 5.413555860519409 [sec]
------------------
lap_normal_time = 1.0775070190429688 [sec]
exp_stats_time = 4.521248817443848 [sec]
exp_shd_time = 1079.2993149757385 [sec]
FPA_time = 0.46752309799194336 [sec]
lap_DFT_time = 1.3767123222351074 [sec]
exp_DFT_time = 5.4705610275268555 [sec]
------------------
lap_normal_time = 1.0671701431274414 [sec]
exp_stats_time = 4.407577037811279 [sec]
exp_shd_time = 1078.0497379302979 [sec]
FPA_time = 0.47409582138061523 [sec]
lap_DFT_time = 1.3690590858459473 [sec]
exp_DFT_time = 5.415027856826782 [sec]
------------------
lap_normal_time = 1.0836942195892334 [sec]
exp_stats_time = 4.439004182815552 [sec]
exp_shd_time = 1090.7139670848846 [sec]
FPA_time = 0.4655637741088867 [sec]
lap_DFT_time = 1.3597121238708496 [sec]
exp_DFT_time = 5.5075581

In [13]:
#K = 5

N = 5000
M = 1000000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(5,5000,1000000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    #print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
#print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 1.0770988464355469 [sec]
exp_stats_time = 6.570231199264526 [sec]
exp_shd_time = 1068.7159218788147 [sec]
FPA_time = 0.45921993255615234 [sec]
lap_DFT_time = 1.2910568714141846 [sec]
exp_DFT_time = 8.103856801986694 [sec]
------------------
lap_normal_time = 1.08538818359375 [sec]
exp_stats_time = 6.579693078994751 [sec]
exp_shd_time = 1056.9886090755463 [sec]
FPA_time = 0.460176944732666 [sec]
lap_DFT_time = 1.2772150039672852 [sec]
exp_DFT_time = 8.11715579032898 [sec]
------------------
lap_normal_time = 1.0789897441864014 [sec]
exp_stats_time = 6.646697044372559 [sec]
exp_shd_time = 1071.9676809310913 [sec]
FPA_time = 0.45436787605285645 [sec]
lap_DFT_time = 1.2781522274017334 [sec]
exp_DFT_time = 8.11932110786438 [sec]
------------------
lap_normal_time = 1.072336196899414 [sec]
exp_stats_time = 6.551396131515503 [sec]
exp_shd_time = 1069.3634538650513 [sec]
FPA_time = 0.47355175018310547 [sec]
lap_DFT_time = 1.3069682121276855 [sec]
exp_DFT_time = 8.145252943038

In [14]:
#K = 10

N = 5000
M = 1000000

ave = np.zeros(7)

for j in range(5):
    RT = runTime(10,5000,1000000)
    ave += RT
    print("lap_normal_time =", RT[0], "[sec]")
    print("exp_stats_time =", RT[1], "[sec]")
    print("exp_shd_time =", RT[2], "[sec]")
    print("FPA_time =", RT[3], "[sec]")
    #print("extended_FPA_time =", RT[4], "[sec]")
    print("lap_DFT_time =", RT[5], "[sec]")
    print("exp_DFT_time =", RT[6], "[sec]")
    print("------------------")

ave /= 5

print("------------------")
print("Average")
print("lap_normal_time =", ave[0], "[sec]")
print("exp_stats_time =", ave[1], "[sec]")
print("exp_shd_time =", ave[2], "[sec]")
print("FPA_time =", ave[3], "[sec]")
#print("extended_FPA_time =", ave[4], "[sec]")
print("lap_DFT_time =", ave[5], "[sec]")
print("exp_DFT_time =", ave[6], "[sec]")

lap_normal_time = 1.083895206451416 [sec]
exp_stats_time = 12.287787675857544 [sec]
exp_shd_time = 1098.3157269954681 [sec]
FPA_time = 0.4575309753417969 [sec]
lap_DFT_time = 1.2322828769683838 [sec]
exp_DFT_time = 14.649231910705566 [sec]
------------------
lap_normal_time = 1.0883049964904785 [sec]
exp_stats_time = 12.387328863143921 [sec]
exp_shd_time = 1070.744609117508 [sec]
FPA_time = 0.46367692947387695 [sec]
lap_DFT_time = 1.2333848476409912 [sec]
exp_DFT_time = 14.750218868255615 [sec]
------------------
lap_normal_time = 1.077651023864746 [sec]
exp_stats_time = 12.590851068496704 [sec]
exp_shd_time = 1052.4194967746735 [sec]
FPA_time = 0.45831775665283203 [sec]
lap_DFT_time = 1.2192950248718262 [sec]
exp_DFT_time = 14.670109987258911 [sec]
------------------
lap_normal_time = 1.102017879486084 [sec]
exp_stats_time = 12.625742197036743 [sec]
exp_shd_time = 1068.6408970355988 [sec]
FPA_time = 0.46708106994628906 [sec]
lap_DFT_time = 1.2273790836334229 [sec]
exp_DFT_time = 14.64